<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Arboles_de_decision_Features_Only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

urltrain = "https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv"
urltest = "https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv"

train = pd.read_csv(urltrain)
test = pd.read_csv(urltest)

train.head()

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [4]:
test.head()

,Unnamed: 0,id,keyword,location,text,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,0,NaN,NaN,Just happened a terrible car crash,34,6,0,0,0,0,0,False,0,0,False,4,False,False
1,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,9,0,0,0,0,0,False,0,0,False,7,False,False
2,2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,19,1,0,0,0,0,False,0,0,False,10,False,False
3,3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,4,1,0,0,0,0,False,0,0,False,3,False,True
4,4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,8,0,0,0,0,0,False,0,0,False,6,True,False


In [5]:
train.columns

Index(['Unnamed: 0', 'id', 'keyword', 'location', 'text', 'target',
       'character_count', 'word_count', 'contains_word_fire',
       'contains_word_storm', 'contains_word_flood', 'contains_word_death',
       'contains_word_love', 'has_mentions', 'amount_of_exclamation_marks',
       'amount_of_interrogation_marks', 'has_link', 'stopwords_count',
       'contains_countries', 'contains_cities'],
      dtype='object')

In [6]:
y_train = train['target']
x_train = train.drop(['Unnamed: 0','id','location','text','target','keyword'], axis = 1)

x_test = test.drop(['Unnamed: 0','id','location','text','keyword'], axis = 1)
x_train.head()

,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [7]:
x_train = x_train*1 #Pongo todos los bool en numeros
x_test = x_test*1

In [8]:
# Saving feature names for later use
feature_list = list(x_train.columns)
# Convert to numpy array
features = np.array(x_train)

labels = np.array(y_train)

In [9]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size = 0.25, random_state = 70)

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 92)

rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=92, verbose=0, warm_start=False)

In [11]:
# Use the forest's predict method on the test data
predictions = rf.predict(val_features)
# Calculate the absolute errors
errors = abs(predictions - val_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.38 degrees.


Usé un regresor, para hacer un submit hay que aproximar si se acerca mas a 0 o a 1

Aqui adelante, uso xgboost

In [12]:
import xgboost as xgb

D_train = xgb.DMatrix(train_features,label = train_labels)
D_val = xgb.DMatrix(val_features, label = val_labels)

#Primeros hiper parámetros de prueba
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

In [13]:
steps = 20

model = xgb.train(param,D_train,steps)

In [14]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_val)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(val_labels, best_preds, average='macro')))
print("Recall = {}".format(recall_score(val_labels, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(val_labels, best_preds)))

Precision = 0.6934711380018959
Recall = 0.6712622549019608
Accuracy = 0.6938025210084033


Medio maleta pero es un primer esqueleto. Otro regresor, de puede transformar a clasificador facilmente

Lgbm lets go!

In [15]:
import lightgbm as lgb

In [22]:
lgb_train = lgb.Dataset(train_features, train_labels)
lgb_eval = lgb.Dataset(val_features,val_labels, reference = lgb_train)

params = {
    'objective':'binary',
    'metric' : 'xentropy',
    'num_leaves' : 30, #Just for trysies
    'learning_rate' : 0.05,
    'feature_fraction': 0.9,
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

#preds = gbm.predict(val_features, num_iteration=gbm.best_iteration)
#rmse = np.sqrt(mean_squared_error(val_labels, preds))
#print("RMSE: %f" % (rmse))

[1]	valid_0's xentropy: 0.674431
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's xentropy: 0.667276
[3]	valid_0's xentropy: 0.660557
[4]	valid_0's xentropy: 0.653984
[5]	valid_0's xentropy: 0.647728
[6]	valid_0's xentropy: 0.642402
[7]	valid_0's xentropy: 0.637266
[8]	valid_0's xentropy: 0.632805
[9]	valid_0's xentropy: 0.62904
[10]	valid_0's xentropy: 0.625298
[11]	valid_0's xentropy: 0.621904
[12]	valid_0's xentropy: 0.618587
[13]	valid_0's xentropy: 0.615576
[14]	valid_0's xentropy: 0.612672
[15]	valid_0's xentropy: 0.610126
[16]	valid_0's xentropy: 0.60808
[17]	valid_0's xentropy: 0.605661
[18]	valid_0's xentropy: 0.603084
[19]	valid_0's xentropy: 0.601286
[20]	valid_0's xentropy: 0.599463
[21]	valid_0's xentropy: 0.598219
[22]	valid_0's xentropy: 0.596742
[23]	valid_0's xentropy: 0.595358
[24]	valid_0's xentropy: 0.593996
[25]	valid_0's xentropy: 0.592856
[26]	valid_0's xentropy: 0.591884
[27]	valid_0's xentropy: 0.591029
[28]	valid_0's xentropy: 0.5901

Es pesimo lgbm pero tengo un template para tunear hyperparámetros. Falta formalizacion del score